In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import warnings
warnings.filterwarnings("ignore")
from performance import performance

In [2]:
a = pd.read_csv('../data/정규/장년_적당_10.csv')

# 정규분포

## 적당 운동 이상치 비율 5%

### 1. 청년 적당

In [3]:
df = pd.read_csv('../data/정규/젊음_적당.csv')
df['update_time'] = pd.to_datetime(a['update_time'])
df.head(2)

,update_time,heart_rate1,event1,heart_rate2,event2,heart_rate3,event3,heart_rate4,event4,heart_rate5,...,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,heart_rate1000,event1000
0,2023-10-01 00:00:00,104,0,164,1,114,0,115,0,152,...,126,0,126,0,90,0,93,0,128,0
1,2023-10-01 00:10:00,99,0,100,0,159,0,145,0,151,...,142,0,125,0,136,0,138,0,135,0


In [4]:
# 최종 결과 데이터 만들기
pred1 = pd.DataFrame()

In [8]:
import logging
logging.getLogger('cmdstanpy').setLevel(logging.CRITICAL)

from prophet import Prophet
for i in range(1, 1001):
    # 필요한 변수만 추출
    df_copy = df[['update_time', f'heart_rate{i}', f'event{i}']].copy()
    # columns 명 변경
    df_copy.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train1 = df_copy[:1008]
    test = df_copy[1008:]
    
    # 필요한 column만 추출
    train = train1[['ds', 'y']]
    test1 = test[['ds']]
    test1.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test1)
    
    # 결과
    result = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    result[f'pred_y{i}'] = result.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred1[f'heart_rate{i}'] = result[['y']]
    pred1[f'event{i}'] = test[[f'event{i}']]
    pred1[f'pred_y{i}'] = result[f'pred_y{i}']

In [9]:
pred1

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,106,0,0,133,0,0,113,0,0,87,...,0,122,0,0,112,0,0,107,0,0
1,128,0,0,125,0,0,103,0,0,111,...,0,117,0,0,143,0,0,138,0,0
2,141,0,0,84,0,0,102,0,0,76,...,0,86,0,0,92,0,0,108,0,0
3,122,0,0,93,0,0,104,0,0,97,...,0,113,0,0,131,0,0,113,0,0
4,154,0,1,102,0,0,107,0,0,132,...,0,134,0,0,135,0,0,111,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,126,0,0,151,0,0,116,0,0,86,...,1,128,0,0,134,0,0,137,0,0
338,108,0,0,118,0,0,136,0,0,114,...,0,142,0,0,111,0,0,102,0,0
339,126,0,0,138,0,0,161,0,1,145,...,0,92,0,0,139,0,0,129,0,0
340,181,1,1,124,0,0,125,0,0,105,...,0,103,0,0,101,0,0,92,0,0


In [10]:
test1.rename(columns = {'ds' : 'update_time'}, inplace = True)

predict1 = pd.concat([test1, pred1], axis = 1)

predict1

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,106,0,0,133,0,0,113,0,0,...,0,122,0,0,112,0,0,107,0,0
1,2023-10-08 00:10:00,128,0,0,125,0,0,103,0,0,...,0,117,0,0,143,0,0,138,0,0
2,2023-10-08 00:20:00,141,0,0,84,0,0,102,0,0,...,0,86,0,0,92,0,0,108,0,0
3,2023-10-08 00:30:00,122,0,0,93,0,0,104,0,0,...,0,113,0,0,131,0,0,113,0,0
4,2023-10-08 00:40:00,154,0,1,102,0,0,107,0,0,...,0,134,0,0,135,0,0,111,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-10-10 08:10:00,126,0,0,151,0,0,116,0,0,...,1,128,0,0,134,0,0,137,0,0
338,2023-10-10 08:20:00,108,0,0,118,0,0,136,0,0,...,0,142,0,0,111,0,0,102,0,0
339,2023-10-10 08:30:00,126,0,0,138,0,0,161,0,1,...,0,92,0,0,139,0,0,129,0,0
340,2023-10-10 08:40:00,181,1,1,124,0,0,125,0,0,...,0,103,0,0,101,0,0,92,0,0


In [11]:
from performance import performance
result = pd.DataFrame()

for i in range(1,1001):

    TP = len(predict1[(predict1[f'event{i}']==1)&(predict1[f'pred_y{i}']==1)])
    TN = len(predict1[(predict1[f'event{i}']==0)&(predict1[f'pred_y{i}']==0)])
    FN = len(predict1[(predict1[f'event{i}']==1)&(predict1[f'pred_y{i}']==0)])
    FP = len(predict1[(predict1[f'event{i}']==0)&(predict1[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN, BETA=1)
    
    result = pd.concat([result, result_i])


result.tail(3)

,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,100.0,99.093656,99.122807,78.571429,100.000000,88.000000
0,95.0,99.378882,99.122807,90.476190,99.688474,92.682927
0,80.0,98.165138,97.368421,66.666667,99.074074,72.727273


In [12]:
청년_적당_5 = pd.DataFrame(index=['mean'])

청년_적당_5['mean_sens'] = result['Sensitivity'].mean()
청년_적당_5['std_sens'] = result['Sensitivity'].std()
청년_적당_5['mean_spec'] = result['Specificity'].mean()
청년_적당_5['std_spec'] = result['Specificity'].std()
청년_적당_5['mean_accu'] = result['Accuracy'].mean()
청년_적당_5['std_accu'] = result['Accuracy'].std()
청년_적당_5['mean_ppv'] = result['PPV'].mean()
청년_적당_5['std_ppv'] = result['PPV'].std()
청년_적당_5['mean_npv'] = result['NPV'].mean()
청년_적당_5['std_npv'] = result['NPV'].std()
청년_적당_5['mean_F1_score'] = result['F_1 score'].mean()
청년_적당_5['std_F1_score'] = result['F_1 score'].std()

청년_적당_5

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,95.383004,7.041537,98.192342,0.936681,98.078655,0.949798,69.482852,12.802262,99.802618,0.296344,79.680877,9.512529


In [13]:
청년_적당_5.to_csv('../prophet_result/정규/5%/청년_적당_5.csv')

### 2. 중년 적당

In [14]:
df = pd.read_csv('../data/정규/중년_적당.csv')
df['update_time'] = pd.to_datetime(df['update_time'])
df.head(2)

,update_time,heart_rate1,event1,heart_rate2,event2,heart_rate3,event3,heart_rate4,event4,heart_rate5,...,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,heart_rate1000,event1000
0,2023-10-01 00:00:00,93,0,149,1,102,0,103,0,138,...,114,0,113,0,79,0,83,0,116,0
1,2023-10-01 00:10:00,88,0,89,0,145,0,132,0,137,...,129,0,113,0,123,0,125,0,122,0


In [15]:
# 최종 결과 데이터 만들기
pred2 = pd.DataFrame()

In [16]:
for i in range(1, 1001):
    # 필요한 변수만 추출
    df2 = df[['update_time', f'heart_rate{i}', f'event{i}']].copy()
    # columns 명 변경
    df2.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train2 = df2[:1008]
    test = df2[1008:]
    
    # 필요한 column만 추출
    train = train2[['ds', 'y']]
    test2 = test[['ds']]
    test2.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test2)
    
    # 결과
    result = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    result[f'pred_y{i}'] = result.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred2[f'heart_rate{i}'] = result[['y']]
    pred2[f'event{i}'] = test[[f'event{i}']]
    pred2[f'pred_y{i}'] = result[f'pred_y{i}']

In [17]:
pred2

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,95,0,0,120,0,0,101,0,0,77,...,0,110,0,0,100,0,0,96,0,0
1,115,0,0,113,0,0,91,0,0,99,...,0,105,0,0,129,0,0,125,0,0
2,128,0,0,74,0,0,91,0,0,66,...,0,76,0,0,82,0,0,97,0,0
3,110,0,0,82,0,0,93,0,0,86,...,0,101,0,0,119,0,0,101,0,0
4,140,0,1,91,0,0,96,0,0,119,...,0,121,0,0,122,0,0,99,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,114,0,0,137,0,0,104,0,0,76,...,1,115,0,0,121,0,0,124,0,0
338,97,0,0,107,0,0,123,0,0,102,...,0,129,0,0,99,0,0,90,0,0
339,114,0,0,125,0,0,147,0,1,132,...,0,82,0,0,126,0,0,116,0,0
340,166,1,1,112,0,0,112,0,0,93,...,0,92,0,0,90,0,0,81,0,0


In [18]:
test2.rename(columns = {'ds' : 'update_time'}, inplace = True)

predict2 = pd.concat([test2, pred2], axis = 1)

predict2

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,95,0,0,120,0,0,101,0,0,...,0,110,0,0,100,0,0,96,0,0
1,2023-10-08 00:10:00,115,0,0,113,0,0,91,0,0,...,0,105,0,0,129,0,0,125,0,0
2,2023-10-08 00:20:00,128,0,0,74,0,0,91,0,0,...,0,76,0,0,82,0,0,97,0,0
3,2023-10-08 00:30:00,110,0,0,82,0,0,93,0,0,...,0,101,0,0,119,0,0,101,0,0
4,2023-10-08 00:40:00,140,0,1,91,0,0,96,0,0,...,0,121,0,0,122,0,0,99,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-10-10 08:10:00,114,0,0,137,0,0,104,0,0,...,1,115,0,0,121,0,0,124,0,0
338,2023-10-10 08:20:00,97,0,0,107,0,0,123,0,0,...,0,129,0,0,99,0,0,90,0,0
339,2023-10-10 08:30:00,114,0,0,125,0,0,147,0,1,...,0,82,0,0,126,0,0,116,0,0
340,2023-10-10 08:40:00,166,1,1,112,0,0,112,0,0,...,0,92,0,0,90,0,0,81,0,0


In [19]:
from performance import performance

result2 = pd.DataFrame()

for i in range(1,1001):

    TP = len(predict2[(predict2[f'event{i}']==1)&(predict2[f'pred_y{i}']==1)])
    TN = len(predict2[(predict2[f'event{i}']==0)&(predict2[f'pred_y{i}']==0)])
    FN = len(predict2[(predict2[f'event{i}']==1)&(predict2[f'pred_y{i}']==0)])
    FP = len(predict2[(predict2[f'event{i}']==0)&(predict2[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN
                           , FP=FP, FN=FN, BETA=1)
    
    result2 = pd.concat([result2, result_i])

print(result2.tail(3))
result2.head(3)

   Sensitivity  Specificity   Accuracy        PPV         NPV  F_1 score
0        100.0    98.795181  98.830409  71.428571  100.000000  83.333333
0         90.0    99.378882  98.830409  90.000000   99.378882  90.000000
0         80.0    98.165138  97.368421  66.666667   99.074074  72.727273


,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,100.0,97.878788,97.953216,63.157895,100.000000,77.419355
0,87.5,98.466258,97.953216,73.684211,99.380805,80.000000
0,100.0,99.093656,99.122807,78.571429,100.000000,88.000000


In [20]:
중년_적당_5 = pd.DataFrame(index=['mean'])

중년_적당_5['mean_sens'] = result2['Sensitivity'].mean()
중년_적당_5['std_sens'] = result2['Sensitivity'].std()
중년_적당_5['mean_spec'] = result2['Specificity'].mean()
중년_적당_5['std_spec'] = result2['Specificity'].std()
중년_적당_5['mean_accu'] = result2['Accuracy'].mean()
중년_적당_5['std_accu'] = result2['Accuracy'].std()
중년_적당_5['mean_ppv'] = result2['PPV'].mean()
중년_적당_5['std_ppv'] = result2['PPV'].std()
중년_적당_5['mean_npv'] = result2['NPV'].mean()
중년_적당_5['std_npv'] = result2['NPV'].std()
중년_적당_5['mean_F1_score'] = result2['F_1 score'].mean()
중년_적당_5['std_F1_score'] = result2['F_1 score'].std()

중년_적당_5

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,95.487063,7.000924,98.187718,0.931441,98.079825,0.935935,69.286442,12.656101,99.809222,0.289297,79.586439,9.42817


In [21]:
중년_적당_5.to_csv('../prophet_result/정규/5%/중년_적당_5.csv')

### 3. 장년 적당

In [22]:
df = pd.read_csv('../data/정규/장년_적당.csv')
df['update_time'] = pd.to_datetime(a['update_time'])
df.head(2)

,update_time,heart_rate1,event1,heart_rate2,event2,heart_rate3,event3,heart_rate4,event4,heart_rate5,...,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,heart_rate1000,event1000
0,2023-10-01 00:00:00,82,0,137,1,91,0,92,0,126,...,102,0,102,0,68,0,72,0,104,0
1,2023-10-01 00:10:00,77,0,78,0,133,0,120,0,125,...,117,0,101,0,111,0,114,0,111,0


In [23]:
# 최종 결과 데이터 만들기
pred3 = pd.DataFrame()

In [29]:
for i in range(1, 1001):
    # 필요한 변수만 추출
    df3 = df[['update_time', f'heart_rate{i}', f'event{i}']].copy()
    # columns 명 변경
    df3.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train3 = df3[:1008]
    test = df3[1008:]
    
    # 필요한 column만 추출
    train = train3[['ds', 'y']]
    test3 = test[['ds']]
    test3.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test3)
    
    # 결과
    result3 = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    result3[f'pred_y{i}'] = result3.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred3[f'heart_rate{i}'] = result3[['y']]
    pred3[f'event{i}'] = test[[f'event{i}']]
    pred3[f'pred_y{i}'] = result3[f'pred_y{i}']

In [30]:
pred3

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,84,0,0,109,0,0,90,0,0,66,...,0,98,0,0,89,0,0,85,0,0
1,104,0,0,101,0,0,80,0,0,88,...,0,94,0,0,117,0,0,113,0,0
2,116,0,0,63,0,0,80,0,0,55,...,0,65,0,0,71,0,0,86,0,0
3,99,0,0,71,0,0,82,0,0,75,...,0,90,0,0,107,0,0,90,0,0
4,128,0,1,80,0,0,84,0,0,107,...,0,109,0,0,110,0,0,88,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,102,0,0,125,0,0,93,0,0,65,...,1,103,0,0,109,0,0,112,0,0
338,86,0,0,95,0,0,111,0,0,91,...,0,117,0,0,88,0,0,79,0,0
339,102,0,0,113,0,0,135,1,1,120,...,0,71,0,0,114,0,0,105,0,0
340,153,1,1,100,0,0,101,0,0,82,...,0,81,0,0,79,0,0,70,0,0


In [34]:
test3.rename(columns = {'ds' : 'update_time'}, inplace = True)

predict3 = pd.concat([test3, pred3], axis = 1)

predict3

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,84,0,0,109,0,0,90,0,0,...,0,98,0,0,89,0,0,85,0,0
1,2023-10-08 00:10:00,104,0,0,101,0,0,80,0,0,...,0,94,0,0,117,0,0,113,0,0
2,2023-10-08 00:20:00,116,0,0,63,0,0,80,0,0,...,0,65,0,0,71,0,0,86,0,0
3,2023-10-08 00:30:00,99,0,0,71,0,0,82,0,0,...,0,90,0,0,107,0,0,90,0,0
4,2023-10-08 00:40:00,128,0,1,80,0,0,84,0,0,...,0,109,0,0,110,0,0,88,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-10-10 08:10:00,102,0,0,125,0,0,93,0,0,...,1,103,0,0,109,0,0,112,0,0
338,2023-10-10 08:20:00,86,0,0,95,0,0,111,0,0,...,0,117,0,0,88,0,0,79,0,0
339,2023-10-10 08:30:00,102,0,0,113,0,0,135,1,1,...,0,71,0,0,114,0,0,105,0,0
340,2023-10-10 08:40:00,153,1,1,100,0,0,101,0,0,...,0,81,0,0,79,0,0,70,0,0


In [35]:
result3 = pd.DataFrame()

for i in range(1,1001):

    TP = len(predict1[(predict1[f'event{i}']==1)&(predict1[f'pred_y{i}']==1)])
    TN = len(predict1[(predict1[f'event{i}']==0)&(predict1[f'pred_y{i}']==0)])
    FN = len(predict1[(predict1[f'event{i}']==1)&(predict1[f'pred_y{i}']==0)])
    FP = len(predict1[(predict1[f'event{i}']==0)&(predict1[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN, BETA=1)
    
    result3 = pd.concat([result3, result_i])


result3.tail(3)

,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,100.0,99.093656,99.122807,78.571429,100.000000,88.000000
0,95.0,99.378882,99.122807,90.476190,99.688474,92.682927
0,80.0,98.165138,97.368421,66.666667,99.074074,72.727273


In [36]:
장년_적당_5 = pd.DataFrame(index=['mean'])

장년_적당_5['mean_sens'] = result3['Sensitivity'].mean()
장년_적당_5['std_sens'] = result3['Sensitivity'].std()
장년_적당_5['mean_spec'] = result3['Specificity'].mean()
장년_적당_5['std_spec'] = result3['Specificity'].std()
장년_적당_5['mean_accu'] = result3['Accuracy'].mean()
장년_적당_5['std_accu'] = result3['Accuracy'].std()
장년_적당_5['mean_ppv'] = result3['PPV'].mean()
장년_적당_5['std_ppv'] = result3['PPV'].std()
장년_적당_5['mean_npv'] = result3['NPV'].mean()
장년_적당_5['std_npv'] = result3['NPV'].std()
장년_적당_5['mean_F1_score'] = result3['F_1 score'].mean()
장년_적당_5['std_F1_score'] = result3['F_1 score'].std()

장년_적당_5

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,95.383004,7.041537,98.192342,0.936681,98.078655,0.949798,69.482852,12.802262,99.802618,0.296344,79.680877,9.512529


In [37]:
장년_적당_5.to_csv('../prophet_result/정규/5%/장년_적당_5.csv')

## 적당 운동 이상치 비율 10%

### 1. 청년 적당

In [38]:
df = pd.read_csv('../data/정규/젊음_적당_10.csv')
df['update_time'] = pd.to_datetime(df['update_time'])
df.head(2)

,update_time,heart_rate1,event1,heart_rate2,event2,heart_rate3,event3,heart_rate4,event4,heart_rate5,...,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,heart_rate1000,event1000
0,2023-10-01 00:00:00,100,0,175,1,113,0,114,0,160,...,128,0,127,0,82,0,87,0,130,0
1,2023-10-01 00:10:00,93,0,95,0,169,1,151,0,159,...,147,0,126,0,140,0,143,0,139,0


In [39]:
# 최종 결과 데이터 만들기
pred1 = pd.DataFrame()

In [40]:
for i in range(1, 1001):
    # 필요한 변수만 추출
    df1 = df[['update_time', f'heart_rate{i}', f'event{i}']].copy()
    # columns 명 변경
    df1.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train1 = df1[:1008]
    test = df1[1008:]
    
    # 필요한 column만 추출
    train = train1[['ds', 'y']]
    test1 = test[['ds']]
    test1.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test1)
    
    # 결과
    result1 = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    result1[f'pred_y{i}'] = result1.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred1[f'heart_rate{i}'] = result1[['y']]
    pred1[f'event{i}'] = test[[f'event{i}']]
    pred1[f'pred_y{i}'] = result1[f'pred_y{i}']

In [41]:
pred1

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,103,0,0,136,0,0,111,0,0,79,...,0,122,0,0,109,0,0,104,0,0
1,130,0,0,127,0,0,98,0,0,108,...,0,117,0,0,148,0,0,142,0,0
2,146,0,0,75,1,0,97,0,0,64,...,0,78,1,0,85,0,0,106,0,0
3,123,0,0,86,0,0,101,0,0,91,...,0,111,0,0,134,0,0,111,0,0
4,162,1,1,97,0,0,104,0,0,135,...,0,137,0,0,139,0,0,108,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,128,0,0,158,0,0,115,0,0,78,...,1,129,0,0,137,0,0,141,0,0
338,105,0,0,118,0,0,139,0,0,112,...,0,148,0,0,108,0,0,97,0,0
339,128,0,0,143,0,0,171,1,1,152,...,0,85,0,0,144,0,0,131,0,0
340,196,1,1,125,0,0,126,0,0,101,...,0,99,0,0,96,0,0,84,0,0


In [42]:
test1.rename(columns = {'ds' : 'update_time'}, inplace = True)

predict1 = pd.concat([test1, pred1], axis = 1)

predict1

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,103,0,0,136,0,0,111,0,0,...,0,122,0,0,109,0,0,104,0,0
1,2023-10-08 00:10:00,130,0,0,127,0,0,98,0,0,...,0,117,0,0,148,0,0,142,0,0
2,2023-10-08 00:20:00,146,0,0,75,1,0,97,0,0,...,0,78,1,0,85,0,0,106,0,0
3,2023-10-08 00:30:00,123,0,0,86,0,0,101,0,0,...,0,111,0,0,134,0,0,111,0,0
4,2023-10-08 00:40:00,162,1,1,97,0,0,104,0,0,...,0,137,0,0,139,0,0,108,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-10-10 08:10:00,128,0,0,158,0,0,115,0,0,...,1,129,0,0,137,0,0,141,0,0
338,2023-10-10 08:20:00,105,0,0,118,0,0,139,0,0,...,0,148,0,0,108,0,0,97,0,0
339,2023-10-10 08:30:00,128,0,0,143,0,0,171,1,1,...,0,85,0,0,144,0,0,131,0,0
340,2023-10-10 08:40:00,196,1,1,125,0,0,126,0,0,...,0,99,0,0,96,0,0,84,0,0


In [43]:
from performance import performance
result1 = pd.DataFrame()

for i in range(1,1001):

    TP = len(predict1[(predict1[f'event{i}']==1)&(predict1[f'pred_y{i}']==1)])
    TN = len(predict1[(predict1[f'event{i}']==0)&(predict1[f'pred_y{i}']==0)])
    FN = len(predict1[(predict1[f'event{i}']==1)&(predict1[f'pred_y{i}']==0)])
    FP = len(predict1[(predict1[f'event{i}']==0)&(predict1[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN, BETA=1)
    
    result1 = pd.concat([result1, result_i])


result1.tail(3)

,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,50.000000,100.000000,95.614035,100.000000,95.412844,66.666667
0,55.263158,100.000000,95.029240,100.000000,94.704050,71.186441
0,53.125000,99.677419,95.321637,94.444444,95.370370,68.000000


In [44]:
청년_적당_10 = pd.DataFrame(index=['mean'])

청년_적당_10['mean_sens'] = result1['Sensitivity'].mean()
청년_적당_10['std_sens'] = result1['Sensitivity'].std()
청년_적당_10['mean_spec'] = result1['Specificity'].mean()
청년_적당_10['std_spec'] = result1['Specificity'].std()
청년_적당_10['mean_accu'] = result1['Accuracy'].mean()
청년_적당_10['std_accu'] = result1['Accuracy'].std()
청년_적당_10['mean_ppv'] = result1['PPV'].mean()
청년_적당_10['std_ppv'] = result1['PPV'].std()
청년_적당_10['mean_npv'] = result1['NPV'].mean()
청년_적당_10['std_npv'] = result1['NPV'].std()
청년_적당_10['mean_F1_score'] = result1['F_1 score'].mean()
청년_적당_10['std_F1_score'] = result1['F_1 score'].std()

청년_적당_10

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,54.529402,10.253468,99.948528,0.178821,95.368713,1.247798,99.280093,2.446967,95.149485,1.30985,69.783107,8.767563


In [45]:
청년_적당_10.to_csv('../prophet_result/정규/10%/청년_적당_10.csv')

### 2. 중년 적당

In [46]:
df = pd.read_csv('../data/정규/중년_적당_10.csv')
df['update_time'] = pd.to_datetime(df['update_time'])
df.head(2)

,update_time,heart_rate1,event1,heart_rate2,event2,heart_rate3,event3,heart_rate4,event4,heart_rate5,...,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,heart_rate1000,event1000
0,2023-10-01 00:00:00,90,0,158,1,101,0,102,0,145,...,115,0,115,0,73,0,77,0,118,0
1,2023-10-01 00:10:00,83,0,85,0,153,1,137,0,143,...,133,0,113,0,126,0,129,0,125,0


In [47]:
# 최종 결과 데이터 만들기
pred2 = pd.DataFrame()

In [48]:
for i in range(1, 1001):
    # 필요한 변수만 추출
    df2 = df[['update_time', f'heart_rate{i}', f'event{i}']].copy()
    # columns 명 변경
    df2.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train2 = df2[:1008]
    test = df2[1008:]
    
    # 필요한 column만 추출
    train = train2[['ds', 'y']]
    test2 = test[['ds']]
    test2.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test2)
    
    # 결과
    result2 = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    result2[f'pred_y{i}'] = result2.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred2[f'heart_rate{i}'] = result2[['y']]
    pred2[f'event{i}'] = test[[f'event{i}']]
    pred2[f'pred_y{i}'] = result2[f'pred_y{i}']

In [49]:
pred2

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,92,0,0,123,0,0,100,0,0,71,...,0,110,0,0,98,0,0,93,0,0
1,117,0,0,114,0,0,88,0,0,97,...,0,105,0,0,134,0,0,129,0,0
2,132,0,0,67,1,0,87,0,0,57,...,0,69,1,0,76,0,0,95,0,0
3,111,0,0,76,0,0,90,0,0,81,...,0,100,0,0,121,0,0,100,0,0
4,147,0,1,87,0,0,93,0,0,122,...,0,124,0,0,125,0,0,97,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,115,0,0,143,0,0,103,0,0,69,...,1,117,0,0,124,0,0,127,0,0
338,94,0,0,106,0,0,126,0,0,101,...,0,134,0,0,97,0,0,87,0,0
339,115,0,0,129,0,0,155,1,1,137,...,0,76,0,0,130,0,0,118,0,0
340,178,1,1,112,0,0,113,0,0,90,...,0,89,0,0,86,0,0,75,0,0


In [50]:
test2.rename(columns = {'ds' : 'update_time'}, inplace = True)

predict2 = pd.concat([test2, pred2], axis = 1)

predict2

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,92,0,0,123,0,0,100,0,0,...,0,110,0,0,98,0,0,93,0,0
1,2023-10-08 00:10:00,117,0,0,114,0,0,88,0,0,...,0,105,0,0,134,0,0,129,0,0
2,2023-10-08 00:20:00,132,0,0,67,1,0,87,0,0,...,0,69,1,0,76,0,0,95,0,0
3,2023-10-08 00:30:00,111,0,0,76,0,0,90,0,0,...,0,100,0,0,121,0,0,100,0,0
4,2023-10-08 00:40:00,147,0,1,87,0,0,93,0,0,...,0,124,0,0,125,0,0,97,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-10-10 08:10:00,115,0,0,143,0,0,103,0,0,...,1,117,0,0,124,0,0,127,0,0
338,2023-10-10 08:20:00,94,0,0,106,0,0,126,0,0,...,0,134,0,0,97,0,0,87,0,0
339,2023-10-10 08:30:00,115,0,0,129,0,0,155,1,1,...,0,76,0,0,130,0,0,118,0,0
340,2023-10-10 08:40:00,178,1,1,112,0,0,113,0,0,...,0,89,0,0,86,0,0,75,0,0


In [51]:
from performance import performance

result2 = pd.DataFrame()

for i in range(1,1001):

    TP = len(predict2[(predict2[f'event{i}']==1)&(predict2[f'pred_y{i}']==1)])
    TN = len(predict2[(predict2[f'event{i}']==0)&(predict2[f'pred_y{i}']==0)])
    FN = len(predict2[(predict2[f'event{i}']==1)&(predict2[f'pred_y{i}']==0)])
    FP = len(predict2[(predict2[f'event{i}']==0)&(predict2[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN
                           , FP=FP, FN=FN, BETA=1)
    
    result2 = pd.concat([result2, result_i])

print(result2.tail(3))
result2.head(3)

   Sensitivity  Specificity   Accuracy         PPV        NPV  F_1 score
0    57.692308   100.000000  96.783626  100.000000  96.636086  73.170732
0    65.625000    99.677419  96.491228   95.454545  96.562500  77.777778
0    60.000000    99.679487  96.198830   94.736842  96.284830  73.469388


,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,59.375000,99.677419,95.906433,95.0,95.962733,73.076923
0,57.575758,100.000000,95.906433,100.0,95.665635,73.076923
0,45.161290,100.000000,95.029240,100.0,94.817073,62.222222


In [52]:
중년_적당_10 = pd.DataFrame(index=['mean'])

중년_적당_10['mean_sens'] = result2['Sensitivity'].mean()
중년_적당_10['std_sens'] = result2['Sensitivity'].std()
중년_적당_10['mean_spec'] = result2['Specificity'].mean()
중년_적당_10['std_spec'] = result2['Specificity'].std()
중년_적당_10['mean_accu'] = result2['Accuracy'].mean()
중년_적당_10['std_accu'] = result2['Accuracy'].std()
중년_적당_10['mean_ppv'] = result2['PPV'].mean()
중년_적당_10['std_ppv'] = result2['PPV'].std()
중년_적당_10['mean_npv'] = result2['NPV'].mean()
중년_적당_10['std_npv'] = result2['NPV'].std()
중년_적당_10['mean_F1_score'] = result2['F_1 score'].mean()
중년_적당_10['std_F1_score'] = result2['F_1 score'].std()

중년_적당_10

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,60.590131,10.646787,99.893768,0.265732,96.364327,1.120013,98.422504,3.802811,96.2568,1.180248,74.392891,8.379335


In [53]:
중년_적당_10.to_csv('../prophet_result/정규/10%/중년_적당_10.csv')

### 3. 장년 적당

In [54]:
df = pd.read_csv('../data/정규/장년_적당_10.csv')
df['update_time'] = pd.to_datetime(df['update_time'])
df.head(2)

,update_time,heart_rate1,event1,heart_rate2,event2,heart_rate3,event3,heart_rate4,event4,heart_rate5,...,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,heart_rate1000,event1000
0,2023-10-01 00:00:00,79,0,146,1,90,0,91,0,132,...,104,0,103,0,62,0,66,0,106,0
1,2023-10-01 00:10:00,72,0,74,0,140,1,124,0,131,...,121,0,102,0,114,0,117,0,114,0


In [55]:
# 최종 결과 데이터 만들기
pred3 = pd.DataFrame()

In [56]:
from prophet import Prophet
for i in range(1, 1001):
    # 필요한 변수만 추출
    df3 = df[['update_time', f'heart_rate{i}', f'event{i}']].copy()
    # columns 명 변경
    df3.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train3 = df3[:1008]
    test = df3[1008:]
    
    # 필요한 column만 추출
    train = train3[['ds', 'y']]
    test3 = test[['ds']]
    test3.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test3)
    
    # 결과
    result3 = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    result3[f'pred_y{i}'] = result3.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred1[f'heart_rate{i}'] = result3[['y']]
    pred1[f'event{i}'] = test[[f'event{i}']]
    pred1[f'pred_y{i}'] = result3[f'pred_y{i}']

In [57]:
pred3

""


In [58]:
test3.rename(columns = {'ds' : 'update_time'}, inplace = True)

predict3 = pd.concat([test3, pred3], axis = 1)

predict3

,update_time
0,2023-10-08 00:00:00
1,2023-10-08 00:10:00
2,2023-10-08 00:20:00
3,2023-10-08 00:30:00
4,2023-10-08 00:40:00
...,...
337,2023-10-10 08:10:00
338,2023-10-10 08:20:00
339,2023-10-10 08:30:00
340,2023-10-10 08:40:00


In [59]:
result3 = pd.DataFrame()

for i in range(1,1001):

    TP = len(predict1[(predict1[f'event{i}']==1)&(predict1[f'pred_y{i}']==1)])
    TN = len(predict1[(predict1[f'event{i}']==0)&(predict1[f'pred_y{i}']==0)])
    FN = len(predict1[(predict1[f'event{i}']==1)&(predict1[f'pred_y{i}']==0)])
    FP = len(predict1[(predict1[f'event{i}']==0)&(predict1[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN, BETA=1)
    
    result3 = pd.concat([result3, result_i])


result3.tail(3)

,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,50.000000,100.000000,95.614035,100.000000,95.412844,66.666667
0,55.263158,100.000000,95.029240,100.000000,94.704050,71.186441
0,53.125000,99.677419,95.321637,94.444444,95.370370,68.000000


In [60]:
장년_적당_10 = pd.DataFrame(index=['mean'])

장년_적당_10['mean_sens'] = result3['Sensitivity'].mean()
장년_적당_10['std_sens'] = result3['Sensitivity'].std()
장년_적당_10['mean_spec'] = result3['Specificity'].mean()
장년_적당_10['std_spec'] = result3['Specificity'].std()
장년_적당_10['mean_accu'] = result3['Accuracy'].mean()
장년_적당_10['std_accu'] = result3['Accuracy'].std()
장년_적당_10['mean_ppv'] = result3['PPV'].mean()
장년_적당_10['std_ppv'] = result3['PPV'].std()
장년_적당_10['mean_npv'] = result3['NPV'].mean()
장년_적당_10['std_npv'] = result3['NPV'].std()
장년_적당_10['mean_F1_score'] = result3['F_1 score'].mean()
장년_적당_10['std_F1_score'] = result3['F_1 score'].std()

장년_적당_10

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,54.529402,10.253468,99.948528,0.178821,95.368713,1.247798,99.280093,2.446967,95.149485,1.30985,69.783107,8.767563


In [61]:
장년_적당_10.to_csv('../prophet_result/정규/10%/장년_적당_10.csv')

# 균등분포

## 적당 운동 이상치 비율 5%

### 1. 청년 적당

In [66]:
df = pd.read_csv('../data/균등/uniform5_청년_적당.csv')
df['update_time'] = pd.to_datetime(a['update_time'])
df.head(2)

,heart_rate1,event1,heart_rate10,event10,heart_rate100,event100,heart_rate1000,event1000,heart_rate101,event101,...,event995,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,update_time
0,103,0,126,0,120,0,199,1,125,0,...,0,109,0,144,0,118,0,150,0,2023-10-01 00:00:00
1,138,0,107,0,119,0,136,0,157,0,...,0,150,0,87,0,118,0,156,0,2023-10-01 00:10:00


In [67]:
# 최종 결과 데이터 만들기
pred1 = pd.DataFrame()

In [68]:
from prophet import Prophet
for i in range(1, 1001):
    # 필요한 변수만 추출
    df_copy = df[['update_time', f'heart_rate{i}', f'event{i}']].copy()
    # columns 명 변경
    df_copy.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train1 = df_copy[:1008]
    test = df_copy[1008:]
    
    # 필요한 column만 추출
    train = train1[['ds', 'y']]
    test1 = test[['ds']]
    test1.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test1)
    
    # 결과
    result1 = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    result1[f'pred_y{i}'] = result1.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred1[f'heart_rate{i}'] = result1[['y']]
    pred1[f'event{i}'] = test[[f'event{i}']]
    pred1[f'pred_y{i}'] = result1[f'pred_y{i}']

In [69]:
pred1

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,127,0,0,112,0,0,158,0,0,149,...,0,139,0,0,85,0,0,109,0,0
1,100,0,0,157,0,0,111,0,0,120,...,0,100,0,0,122,0,0,160,0,0
2,141,0,0,119,0,0,158,0,0,203,...,0,154,0,0,149,0,0,159,0,0
3,131,0,0,134,0,0,109,0,0,156,...,0,97,0,0,106,0,0,99,0,0
4,140,0,0,116,0,0,152,0,0,127,...,0,148,0,0,123,0,0,126,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,209,1,1,95,0,0,139,0,0,131,...,0,150,0,0,125,0,0,125,0,0
338,112,0,0,104,0,0,97,0,0,157,...,0,88,0,0,119,0,0,140,0,0
339,134,0,0,153,0,0,113,0,0,140,...,0,160,0,0,191,1,1,84,0,0
340,85,0,0,92,0,0,85,0,0,93,...,0,80,0,0,94,0,0,110,0,0


In [70]:
test1.rename(columns = {'ds' : 'update_time'}, inplace = True)

predict1 = pd.concat([test1, pred1], axis = 1)

predict1

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,127,0,0,112,0,0,158,0,0,...,0,139,0,0,85,0,0,109,0,0
1,2023-10-08 00:10:00,100,0,0,157,0,0,111,0,0,...,0,100,0,0,122,0,0,160,0,0
2,2023-10-08 00:20:00,141,0,0,119,0,0,158,0,0,...,0,154,0,0,149,0,0,159,0,0
3,2023-10-08 00:30:00,131,0,0,134,0,0,109,0,0,...,0,97,0,0,106,0,0,99,0,0
4,2023-10-08 00:40:00,140,0,0,116,0,0,152,0,0,...,0,148,0,0,123,0,0,126,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-10-10 08:10:00,209,1,1,95,0,0,139,0,0,...,0,150,0,0,125,0,0,125,0,0
338,2023-10-10 08:20:00,112,0,0,104,0,0,97,0,0,...,0,88,0,0,119,0,0,140,0,0
339,2023-10-10 08:30:00,134,0,0,153,0,0,113,0,0,...,0,160,0,0,191,1,1,84,0,0
340,2023-10-10 08:40:00,85,0,0,92,0,0,85,0,0,...,0,80,0,0,94,0,0,110,0,0


In [71]:
from performance import performance
result1 = pd.DataFrame()

for i in range(1,1001):

    TP = len(predict1[(predict1[f'event{i}']==1)&(predict1[f'pred_y{i}']==1)])
    TN = len(predict1[(predict1[f'event{i}']==0)&(predict1[f'pred_y{i}']==0)])
    FN = len(predict1[(predict1[f'event{i}']==1)&(predict1[f'pred_y{i}']==0)])
    FP = len(predict1[(predict1[f'event{i}']==0)&(predict1[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN, BETA=1)
    
    result1 = pd.concat([result1, result_i])


result1.tail(3)

,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,100.0,100.0,100.000000,100.0,100.000000,100.000000
0,90.0,100.0,99.415205,100.0,99.382716,94.736842
0,100.0,100.0,100.000000,100.0,100.000000,100.000000


In [72]:
청년_적당_5_균등 = pd.DataFrame(index=['mean'])

청년_적당_5_균등['mean_sens'] = result1['Sensitivity'].mean()
청년_적당_5_균등['std_sens'] = result1['Sensitivity'].std()
청년_적당_5_균등['mean_spec'] = result1['Specificity'].mean()
청년_적당_5_균등['std_spec'] = result1['Specificity'].std()
청년_적당_5_균등['mean_accu'] = result1['Accuracy'].mean()
청년_적당_5_균등['std_accu'] = result1['Accuracy'].std()
청년_적당_5_균등['mean_ppv'] = result1['PPV'].mean()
청년_적당_5_균등['std_ppv'] = result1['PPV'].std()
청년_적당_5_균등['mean_npv'] = result1['NPV'].mean()
청년_적당_5_균등['std_npv'] = result1['NPV'].std()
청년_적당_5_균등['mean_F1_score'] = result1['F_1 score'].mean()
청년_적당_5_균등['std_F1_score'] = result1['F_1 score'].std()

청년_적당_5_균등

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,97.766574,4.620167,99.992876,0.062334,99.887719,0.234865,99.869098,1.171215,99.8895,0.219964,98.752279,2.742864


In [73]:
청년_적당_5.to_csv('../prophet_result/균등/청년_적당_5.csv')

### 2. 중년 적당

In [76]:
df = pd.read_csv('../data/균등/uniform5_중년_적당.csv')
df['update_time'] = pd.to_datetime(a['update_time'])
df.head(2)

,heart_rate1,event1,heart_rate10,event10,heart_rate100,event100,heart_rate1000,event1000,heart_rate101,event101,...,event995,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,update_time
0,145,0,106,0,128,0,139,0,108,0,...,1,121,0,105,0,123,0,181,1,2023-10-01 00:00:00
1,128,0,91,0,90,0,99,0,134,0,...,0,107,0,97,0,124,0,131,0,2023-10-01 00:10:00


In [77]:
# 최종 결과 데이터 만들기
pred2 = pd.DataFrame()

In [78]:
from prophet import Prophet
for i in range(1, 1001):
    # 필요한 변수만 추출
    df_copy = df[['update_time', f'heart_rate{i}', f'event{i}']].copy()
    # columns 명 변경
    df_copy.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train2 = df_copy[:1008]
    test = df_copy[1008:]
    
    # 필요한 column만 추출
    train = train2[['ds', 'y']]
    test2 = test[['ds']]
    test2.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test2)
    
    # 결과
    result2 = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    result2[f'pred_y{i}'] = result2.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred2[f'heart_rate{i}'] = result2[['y']]
    pred2[f'event{i}'] = test[[f'event{i}']]
    pred2[f'pred_y{i}'] = result2[f'pred_y{i}']

In [82]:
pred2

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,99,0,0,190,1,1,126,0,0,94,...,0,113,0,0,116,0,0,83,0,0
1,77,0,0,143,0,0,88,0,0,73,...,0,115,0,0,101,0,0,128,0,0
2,101,0,0,98,0,0,143,0,0,124,...,0,108,0,0,133,0,0,99,0,0
3,140,0,0,98,0,0,119,0,0,76,...,1,138,0,0,48,1,1,117,0,0
4,103,0,0,111,0,0,24,1,1,93,...,0,99,0,0,132,0,0,140,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,147,0,0,110,0,0,86,0,0,85,...,0,105,0,0,137,0,0,115,0,0
338,144,0,0,183,1,1,142,0,0,85,...,0,114,0,0,97,0,0,139,0,0
339,111,0,0,107,0,0,117,0,0,142,...,0,74,0,0,132,0,0,81,0,0
340,142,0,0,136,0,0,129,0,0,82,...,0,117,0,0,178,1,1,145,0,0


In [83]:
test2.rename(columns = {'ds' : 'update_time'}, inplace = True)

predict2 = pd.concat([test2, pred2], axis = 1)

predict2

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,99,0,0,190,1,1,126,0,0,...,0,113,0,0,116,0,0,83,0,0
1,2023-10-08 00:10:00,77,0,0,143,0,0,88,0,0,...,0,115,0,0,101,0,0,128,0,0
2,2023-10-08 00:20:00,101,0,0,98,0,0,143,0,0,...,0,108,0,0,133,0,0,99,0,0
3,2023-10-08 00:30:00,140,0,0,98,0,0,119,0,0,...,1,138,0,0,48,1,1,117,0,0
4,2023-10-08 00:40:00,103,0,0,111,0,0,24,1,1,...,0,99,0,0,132,0,0,140,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-10-10 08:10:00,147,0,0,110,0,0,86,0,0,...,0,105,0,0,137,0,0,115,0,0
338,2023-10-10 08:20:00,144,0,0,183,1,1,142,0,0,...,0,114,0,0,97,0,0,139,0,0
339,2023-10-10 08:30:00,111,0,0,107,0,0,117,0,0,...,0,74,0,0,132,0,0,81,0,0
340,2023-10-10 08:40:00,142,0,0,136,0,0,129,0,0,...,0,117,0,0,178,1,1,145,0,0


In [84]:
from performance import performance
result2 = pd.DataFrame()

for i in range(1,1001):

    TP = len(predict2[(predict2[f'event{i}']==1)&(predict2[f'pred_y{i}']==1)])
    TN = len(predict2[(predict2[f'event{i}']==0)&(predict2[f'pred_y{i}']==0)])
    FN = len(predict2[(predict2[f'event{i}']==1)&(predict2[f'pred_y{i}']==0)])
    FP = len(predict2[(predict2[f'event{i}']==0)&(predict2[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN, BETA=1)
    
    result2 = pd.concat([result2, result_i])


result2.tail(3)

,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,100.000000,100.0,100.000000,100.0,100.000000,100.000000
0,90.909091,100.0,99.707602,100.0,99.698795,95.238095
0,100.000000,100.0,100.000000,100.0,100.000000,100.000000


In [85]:
중년_적당_균등_5 = pd.DataFrame(index=['mean'])

중년_적당_균등_5['mean_sens'] = result2['Sensitivity'].mean()
중년_적당_균등_5['std_sens'] = result2['Sensitivity'].std()
중년_적당_균등_5['mean_spec'] = result2['Specificity'].mean()
중년_적당_균등_5['std_spec'] = result2['Specificity'].std()
중년_적당_균등_5['mean_accu'] = result2['Accuracy'].mean()
중년_적당_균등_5['std_accu'] = result2['Accuracy'].std()
중년_적당_균등_5['mean_ppv'] = result2['PPV'].mean()
중년_적당_균등_5['std_ppv'] = result2['PPV'].std()
중년_적당_균등_5['mean_npv'] = result2['NPV'].mean()
중년_적당_균등_5['std_npv'] = result2['NPV'].std()
중년_적당_균등_5['mean_F1_score'] = result2['F_1 score'].mean()
중년_적당_균등_5['std_F1_score'] = result2['F_1 score'].std()

중년_적당_균등_5

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,99.437809,2.13378,99.99785,0.029048,99.97193,0.101671,99.958665,0.570709,99.972764,0.10006,99.685224,1.18895


In [86]:
중년_적당_균등_5.to_csv('../prophet_result/균등/중년_적당_균등_5.csv')

### 3. 장년 적당

In [87]:
df = pd.read_csv('../data/균등/uniform5_노년_적당.csv')
df['update_time'] = pd.to_datetime(a['update_time'])
df.head(2)

,heart_rate1,event1,heart_rate10,event10,heart_rate100,event100,heart_rate1000,event1000,heart_rate101,event101,...,event995,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,update_time
0,95,0,109,0,128,0,87,0,94,0,...,0,124,0,75,0,91,0,102,0,2023-10-01 00:00:00
1,118,0,128,0,69,0,82,0,108,0,...,0,80,0,73,0,155,1,131,0,2023-10-01 00:10:00


In [88]:
# 최종 결과 데이터 만들기
pred3 = pd.DataFrame()

In [89]:
from prophet import Prophet
for i in range(1, 1001):
    # 필요한 변수만 추출
    df_copy = df[['update_time', f'heart_rate{i}', f'event{i}']].copy()
    # columns 명 변경
    df_copy.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train3 = df_copy[:1008]
    test = df_copy[1008:]
    
    # 필요한 column만 추출
    train = train3[['ds', 'y']]
    test3 = test[['ds']]
    test3.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test3)
    
    # 결과
    result3 = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    result3[f'pred_y{i}'] = result3.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred3[f'heart_rate{i}'] = result3[['y']]
    pred3[f'event{i}'] = test[[f'event{i}']]
    pred3[f'pred_y{i}'] = result3[f'pred_y{i}']

In [90]:
pred3

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,84,0,0,103,0,0,121,0,0,92,...,0,130,0,0,62,0,0,80,0,0
1,132,0,0,76,0,0,96,0,0,102,...,0,68,0,0,90,0,0,85,0,0
2,83,0,0,120,0,0,90,0,0,72,...,0,69,0,0,93,0,0,111,0,0
3,124,0,0,117,0,0,74,0,0,132,...,0,76,0,0,121,0,0,69,0,0
4,82,0,0,66,0,0,133,0,0,132,...,0,116,0,0,112,0,0,62,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,123,0,0,83,0,0,72,0,0,119,...,1,68,0,0,124,0,0,118,0,0
338,84,0,0,91,0,0,130,0,0,124,...,0,127,0,0,67,0,0,97,0,0
339,129,0,0,124,0,0,179,1,1,173,...,1,133,0,0,123,0,0,100,0,0
340,76,0,0,113,0,0,105,0,0,107,...,0,86,0,0,106,0,0,104,0,0


In [91]:
test3.rename(columns = {'ds' : 'update_time'}, inplace = True)

predict3 = pd.concat([test3, pred3], axis = 1)

predict3

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,84,0,0,103,0,0,121,0,0,...,0,130,0,0,62,0,0,80,0,0
1,2023-10-08 00:10:00,132,0,0,76,0,0,96,0,0,...,0,68,0,0,90,0,0,85,0,0
2,2023-10-08 00:20:00,83,0,0,120,0,0,90,0,0,...,0,69,0,0,93,0,0,111,0,0
3,2023-10-08 00:30:00,124,0,0,117,0,0,74,0,0,...,0,76,0,0,121,0,0,69,0,0
4,2023-10-08 00:40:00,82,0,0,66,0,0,133,0,0,...,0,116,0,0,112,0,0,62,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-10-10 08:10:00,123,0,0,83,0,0,72,0,0,...,1,68,0,0,124,0,0,118,0,0
338,2023-10-10 08:20:00,84,0,0,91,0,0,130,0,0,...,0,127,0,0,67,0,0,97,0,0
339,2023-10-10 08:30:00,129,0,0,124,0,0,179,1,1,...,1,133,0,0,123,0,0,100,0,0
340,2023-10-10 08:40:00,76,0,0,113,0,0,105,0,0,...,0,86,0,0,106,0,0,104,0,0


In [92]:
from performance import performance
result3 = pd.DataFrame()

for i in range(1,1001):

    TP = len(predict3[(predict3[f'event{i}']==1)&(predict3[f'pred_y{i}']==1)])
    TN = len(predict3[(predict3[f'event{i}']==0)&(predict3[f'pred_y{i}']==0)])
    FN = len(predict3[(predict3[f'event{i}']==1)&(predict3[f'pred_y{i}']==0)])
    FP = len(predict3[(predict3[f'event{i}']==0)&(predict3[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN, BETA=1)
    
    result3 = pd.concat([result3, result_i])


result3.tail(3)

,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,100.0,100.0,100.0,100.0,100.0,100.0
0,100.0,100.0,100.0,100.0,100.0,100.0
0,100.0,100.0,100.0,100.0,100.0,100.0


In [93]:
장년_적당_5_균등 = pd.DataFrame(index=['mean'])

장년_적당_5_균등['mean_sens'] = result3['Sensitivity'].mean()
장년_적당_5_균등['std_sens'] = result3['Sensitivity'].std()
장년_적당_5_균등['mean_spec'] = result3['Specificity'].mean()
장년_적당_5_균등['std_spec'] = result3['Specificity'].std()
장년_적당_5_균등['mean_accu'] = result3['Accuracy'].mean()
장년_적당_5_균등['std_accu'] = result3['Accuracy'].std()
장년_적당_5_균등['mean_ppv'] = result3['PPV'].mean()
장년_적당_5_균등['std_ppv'] = result3['PPV'].std()
장년_적당_5_균등['mean_npv'] = result3['NPV'].mean()
장년_적당_5_균등['std_npv'] = result3['NPV'].std()
장년_적당_5_균등['mean_F1_score'] = result3['F_1 score'].mean()
장년_적당_5_균등['std_F1_score'] = result3['F_1 score'].std()

장년_적당_5_균등

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,96.732115,5.290158,99.986762,0.131989,99.830994,0.317773,99.748532,2.422642,99.836185,0.263421,98.150344,3.48001


In [94]:
장년_적당_5_균등.to_csv('../prophet_result/균등/장년_적당_5_균등.csv')

## 적당 운동 이상치 비율 10%

### 1. 청년 적당

In [95]:
df = pd.read_csv('../data/균등/uniform10_청년_적당.csv')
df['update_time'] = pd.to_datetime(a['update_time'])
df.head(2)

,heart_rate1,event1,heart_rate10,event10,heart_rate100,event100,heart_rate1000,event1000,heart_rate101,event101,...,event995,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,update_time
0,94,0,89,0,49,1,137,0,148,0,...,0,131,0,84,0,140,0,111,0,2023-10-01 00:00:00
1,209,1,83,0,129,0,115,0,102,0,...,0,108,0,127,0,119,0,48,1,2023-10-01 00:10:00


In [96]:
# 최종 결과 데이터 만들기
pred1 = pd.DataFrame()

In [97]:
from prophet import Prophet
for i in range(1, 1001):
    # 필요한 변수만 추출
    df_copy = df[['update_time', f'heart_rate{i}', f'event{i}']].copy()
    # columns 명 변경
    df_copy.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train1 = df_copy[:1008]
    test = df_copy[1008:]
    
    # 필요한 column만 추출
    train = train1[['ds', 'y']]
    test1 = test[['ds']]
    test1.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test1)
    
    # 결과
    result1 = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    result1[f'pred_y{i}'] = result1.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred1[f'heart_rate{i}'] = result1[['y']]
    pred1[f'event{i}'] = test[[f'event{i}']]
    pred1[f'pred_y{i}'] = result1[f'pred_y{i}']

In [98]:
pred1

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,122,0,0,153,0,0,87,0,0,93,...,0,197,1,1,125,0,0,181,1,0
1,103,0,0,155,0,0,82,0,0,147,...,0,108,0,0,102,0,0,142,0,0
2,95,0,0,108,0,0,136,0,0,184,...,1,82,0,0,185,1,0,134,0,0
3,91,0,0,128,0,0,121,0,0,141,...,1,153,0,0,139,0,0,153,0,0
4,133,0,0,43,1,1,203,1,1,126,...,0,138,0,0,87,0,0,143,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,95,0,0,97,0,0,83,0,0,108,...,0,139,0,0,134,0,0,115,0,0
338,92,0,0,138,0,0,39,1,1,103,...,0,182,1,0,185,1,1,98,0,0
339,90,0,0,139,0,0,129,0,0,153,...,0,117,0,0,147,0,0,127,0,0
340,205,1,1,137,0,0,117,0,0,82,...,0,97,0,0,132,0,0,118,0,0


In [99]:
test1.rename(columns = {'ds' : 'update_time'}, inplace = True)

predict1 = pd.concat([test1, pred1], axis = 1)

predict1

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,122,0,0,153,0,0,87,0,0,...,0,197,1,1,125,0,0,181,1,0
1,2023-10-08 00:10:00,103,0,0,155,0,0,82,0,0,...,0,108,0,0,102,0,0,142,0,0
2,2023-10-08 00:20:00,95,0,0,108,0,0,136,0,0,...,1,82,0,0,185,1,0,134,0,0
3,2023-10-08 00:30:00,91,0,0,128,0,0,121,0,0,...,1,153,0,0,139,0,0,153,0,0
4,2023-10-08 00:40:00,133,0,0,43,1,1,203,1,1,...,0,138,0,0,87,0,0,143,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-10-10 08:10:00,95,0,0,97,0,0,83,0,0,...,0,139,0,0,134,0,0,115,0,0
338,2023-10-10 08:20:00,92,0,0,138,0,0,39,1,1,...,0,182,1,0,185,1,1,98,0,0
339,2023-10-10 08:30:00,90,0,0,139,0,0,129,0,0,...,0,117,0,0,147,0,0,127,0,0
340,2023-10-10 08:40:00,205,1,1,137,0,0,117,0,0,...,0,97,0,0,132,0,0,118,0,0


In [100]:
from performance import performance
result1 = pd.DataFrame()

for i in range(1,1001):

    TP = len(predict1[(predict1[f'event{i}']==1)&(predict1[f'pred_y{i}']==1)])
    TN = len(predict1[(predict1[f'event{i}']==0)&(predict1[f'pred_y{i}']==0)])
    FN = len(predict1[(predict1[f'event{i}']==1)&(predict1[f'pred_y{i}']==0)])
    FP = len(predict1[(predict1[f'event{i}']==0)&(predict1[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN, BETA=1)
    
    result1 = pd.concat([result1, result_i])


result1.tail(3)

,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,87.878788,100.0,98.830409,100.0,98.722045,93.548387
0,85.294118,100.0,98.538012,100.0,98.402556,92.063492
0,56.000000,100.0,96.783626,100.0,96.646341,71.794872


In [102]:
청년_적당_10_균등 = pd.DataFrame(index=['mean'])

청년_적당_10_균등['mean_sens'] = result1['Sensitivity'].mean()
청년_적당_10_균등['std_sens'] = result1['Sensitivity'].std()
청년_적당_10_균등['mean_spec'] = result1['Specificity'].mean()
청년_적당_10_균등['std_spec'] = result1['Specificity'].std()
청년_적당_10_균등['mean_accu'] = result1['Accuracy'].mean()
청년_적당_10_균등['std_accu'] = result1['Accuracy'].std()
청년_적당_10_균등['mean_ppv'] = result1['PPV'].mean()
청년_적당_10_균등['std_ppv'] = result1['PPV'].std()
청년_적당_10_균등['mean_npv'] = result1['NPV'].mean()
청년_적당_10_균등['std_npv'] = result1['NPV'].std()
청년_적당_10_균등['mean_F1_score'] = result1['F_1 score'].mean()
청년_적당_10_균등['std_F1_score'] = result1['F_1 score'].std()

청년_적당_10_균등

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,79.157041,10.007821,100.0,0.0,97.969591,0.941104,100.0,0.0,97.805066,0.998148,88.005184,6.48773


In [103]:
청년_적당_10_균등.to_csv('../prophet_result/균등/10%/청년_적당_10_균등.csv')

### 2. 중년 적당

In [104]:
df = pd.read_csv('../data/균등/uniform10_중년_적당.csv')
df['update_time'] = pd.to_datetime(a['update_time'])
df.head(2)

,heart_rate1,event1,heart_rate10,event10,heart_rate100,event100,heart_rate1000,event1000,heart_rate101,event101,...,event995,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,update_time
0,73,0,127,0,98,0,87,0,172,1,...,0,139,0,79,0,114,0,75,0,2023-10-01 00:00:00
1,120,0,119,0,141,0,78,0,26,1,...,0,88,0,94,0,145,0,128,0,2023-10-01 00:10:00


In [105]:
# 최종 결과 데이터 만들기
pred2 = pd.DataFrame()

In [106]:
from prophet import Prophet
for i in range(1, 1001):
    # 필요한 변수만 추출
    df_copy = df[['update_time', f'heart_rate{i}', f'event{i}']].copy()
    # columns 명 변경
    df_copy.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train2 = df_copy[:1008]
    test = df_copy[1008:]
    
    # 필요한 column만 추출
    train = train2[['ds', 'y']]
    test2 = test[['ds']]
    test2.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test2)
    
    # 결과
    result2 = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    result2[f'pred_y{i}'] = result2.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred2[f'heart_rate{i}'] = result2[['y']]
    pred2[f'event{i}'] = test[[f'event{i}']]
    pred2[f'pred_y{i}'] = result2[f'pred_y{i}']

In [107]:
pred2

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,93,0,0,110,0,0,135,0,0,134,...,0,74,0,0,96,0,0,71,0,0
1,116,0,0,90,0,0,97,0,0,140,...,0,115,0,0,92,0,0,112,0,0
2,107,0,0,72,0,0,142,0,0,98,...,0,86,0,0,88,0,0,197,1,1
3,97,0,0,140,0,0,99,0,0,91,...,0,106,0,0,80,0,0,137,0,0
4,136,0,0,143,0,0,146,0,0,85,...,0,94,0,0,192,1,1,139,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,127,0,0,79,0,0,141,0,0,140,...,0,77,0,0,123,0,0,131,0,0
338,146,0,0,111,0,0,72,0,0,91,...,0,93,0,0,141,0,0,101,0,0
339,186,1,1,141,0,0,131,0,0,141,...,0,111,0,0,104,0,0,176,1,1
340,131,0,0,91,0,0,81,0,0,140,...,0,133,0,0,119,0,0,178,1,1


In [108]:
test2.rename(columns = {'ds' : 'update_time'}, inplace = True)

predict2 = pd.concat([test2, pred2], axis = 1)

predict2

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,93,0,0,110,0,0,135,0,0,...,0,74,0,0,96,0,0,71,0,0
1,2023-10-08 00:10:00,116,0,0,90,0,0,97,0,0,...,0,115,0,0,92,0,0,112,0,0
2,2023-10-08 00:20:00,107,0,0,72,0,0,142,0,0,...,0,86,0,0,88,0,0,197,1,1
3,2023-10-08 00:30:00,97,0,0,140,0,0,99,0,0,...,0,106,0,0,80,0,0,137,0,0
4,2023-10-08 00:40:00,136,0,0,143,0,0,146,0,0,...,0,94,0,0,192,1,1,139,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-10-10 08:10:00,127,0,0,79,0,0,141,0,0,...,0,77,0,0,123,0,0,131,0,0
338,2023-10-10 08:20:00,146,0,0,111,0,0,72,0,0,...,0,93,0,0,141,0,0,101,0,0
339,2023-10-10 08:30:00,186,1,1,141,0,0,131,0,0,...,0,111,0,0,104,0,0,176,1,1
340,2023-10-10 08:40:00,131,0,0,91,0,0,81,0,0,...,0,133,0,0,119,0,0,178,1,1


In [109]:
from performance import performance
result2 = pd.DataFrame()

for i in range(1,1001):

    TP = len(predict2[(predict2[f'event{i}']==1)&(predict2[f'pred_y{i}']==1)])
    TN = len(predict2[(predict2[f'event{i}']==0)&(predict2[f'pred_y{i}']==0)])
    FN = len(predict2[(predict2[f'event{i}']==1)&(predict2[f'pred_y{i}']==0)])
    FP = len(predict2[(predict2[f'event{i}']==0)&(predict2[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN, BETA=1)
    
    result2 = pd.concat([result2, result_i])


result2.tail(3)

,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,80.645161,100.0,98.245614,100.0,98.107256,89.285714
0,93.939394,100.0,99.415205,100.0,99.356913,96.875000
0,80.769231,100.0,98.538012,100.0,98.442368,89.361702


In [110]:
중년_적당_균등_10 = pd.DataFrame(index=['mean'])

중년_적당_균등_10['mean_sens'] = result2['Sensitivity'].mean()
중년_적당_균등_10['std_sens'] = result2['Sensitivity'].std()
중년_적당_균등_10['mean_spec'] = result2['Specificity'].mean()
중년_적당_균등_10['std_spec'] = result2['Specificity'].std()
중년_적당_균등_10['mean_accu'] = result2['Accuracy'].mean()
중년_적당_균등_10['std_accu'] = result2['Accuracy'].std()
중년_적당_균등_10['mean_ppv'] = result2['PPV'].mean()
중년_적당_균등_10['std_ppv'] = result2['PPV'].std()
중년_적당_균등_10['mean_npv'] = result2['NPV'].mean()
중년_적당_균등_10['std_npv'] = result2['NPV'].std()
중년_적당_균등_10['mean_F1_score'] = result2['F_1 score'].mean()
중년_적당_균등_10['std_F1_score'] = result2['F_1 score'].std()

중년_적당_균등_10

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,86.247359,7.817392,100.0,0.0,98.64386,0.755362,100.0,0.0,98.520832,0.814147,92.421613,4.641695


In [111]:
중년_적당_균등_10.to_csv('../prophet_result/균등/10%/중년_적당_균등_10.csv')

### 3. 장년 적당

In [112]:
df = pd.read_csv('../data/균등/uniform10_노년_적당.csv')
df['update_time'] = pd.to_datetime(a['update_time'])
df.head(2)

,heart_rate1,event1,heart_rate10,event10,heart_rate100,event100,heart_rate1000,event1000,heart_rate101,event101,...,event995,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,update_time
0,64,0,113,0,99,0,124,0,74,0,...,0,166,1,117,0,109,0,29,1,2023-10-01 00:00:00
1,179,1,133,0,158,1,120,0,173,1,...,0,104,0,99,0,99,0,81,0,2023-10-01 00:10:00


In [113]:
# 최종 결과 데이터 만들기
pred3 = pd.DataFrame()

In [114]:
from prophet import Prophet
for i in range(1, 1001):
    # 필요한 변수만 추출
    df_copy = df[['update_time', f'heart_rate{i}', f'event{i}']].copy()
    # columns 명 변경
    df_copy.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train3 = df_copy[:1008]
    test = df_copy[1008:]
    
    # 필요한 column만 추출
    train = train3[['ds', 'y']]
    test3 = test[['ds']]
    test3.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test3)
    
    # 결과
    result3 = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    result3[f'pred_y{i}'] = result3.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred3[f'heart_rate{i}'] = result3[['y']]
    pred3[f'event{i}'] = test[[f'event{i}']]
    pred3[f'pred_y{i}'] = result3[f'pred_y{i}']

In [115]:
pred3

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,118,0,0,99,0,0,131,0,0,122,...,0,92,0,0,125,0,0,112,0,0
1,129,0,0,119,0,0,133,0,0,93,...,0,70,0,0,99,0,0,98,0,0
2,131,0,0,76,0,0,161,1,1,61,...,0,104,0,0,118,0,0,94,0,0
3,86,0,0,87,0,0,105,0,0,108,...,0,62,0,0,66,0,0,127,0,0
4,176,1,1,82,0,0,128,0,0,66,...,0,91,0,0,111,0,0,127,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,93,0,0,130,0,0,67,0,0,116,...,0,99,0,0,102,0,0,112,0,0
338,92,0,0,125,0,0,76,0,0,121,...,1,101,0,0,69,0,0,98,0,0
339,164,1,1,97,0,0,109,0,0,99,...,0,130,0,0,76,0,0,103,0,0
340,97,0,0,82,0,0,110,0,0,76,...,0,75,0,0,96,0,0,104,0,0


In [116]:
test3.rename(columns = {'ds' : 'update_time'}, inplace = True)

predict3 = pd.concat([test3, pred3], axis = 1)

predict3

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,118,0,0,99,0,0,131,0,0,...,0,92,0,0,125,0,0,112,0,0
1,2023-10-08 00:10:00,129,0,0,119,0,0,133,0,0,...,0,70,0,0,99,0,0,98,0,0
2,2023-10-08 00:20:00,131,0,0,76,0,0,161,1,1,...,0,104,0,0,118,0,0,94,0,0
3,2023-10-08 00:30:00,86,0,0,87,0,0,105,0,0,...,0,62,0,0,66,0,0,127,0,0
4,2023-10-08 00:40:00,176,1,1,82,0,0,128,0,0,...,0,91,0,0,111,0,0,127,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-10-10 08:10:00,93,0,0,130,0,0,67,0,0,...,0,99,0,0,102,0,0,112,0,0
338,2023-10-10 08:20:00,92,0,0,125,0,0,76,0,0,...,1,101,0,0,69,0,0,98,0,0
339,2023-10-10 08:30:00,164,1,1,97,0,0,109,0,0,...,0,130,0,0,76,0,0,103,0,0
340,2023-10-10 08:40:00,97,0,0,82,0,0,110,0,0,...,0,75,0,0,96,0,0,104,0,0


In [117]:
from performance import performance
result3 = pd.DataFrame()

for i in range(1,1001):

    TP = len(predict3[(predict3[f'event{i}']==1)&(predict3[f'pred_y{i}']==1)])
    TN = len(predict3[(predict3[f'event{i}']==0)&(predict3[f'pred_y{i}']==0)])
    FN = len(predict3[(predict3[f'event{i}']==1)&(predict3[f'pred_y{i}']==0)])
    FP = len(predict3[(predict3[f'event{i}']==0)&(predict3[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN, BETA=1)
    
    result3 = pd.concat([result3, result_i])


result3.tail(3)

,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,77.419355,100.0,97.953216,100.0,97.798742,87.272727
0,76.666667,100.0,97.953216,100.0,97.805643,86.792453
0,81.081081,100.0,97.953216,100.0,97.756410,89.552239


In [118]:
장년_적당_10_균등 = pd.DataFrame(index=['mean'])

장년_적당_10_균등['mean_sens'] = result3['Sensitivity'].mean()
장년_적당_10_균등['std_sens'] = result3['Sensitivity'].std()
장년_적당_10_균등['mean_spec'] = result3['Specificity'].mean()
장년_적당_10_균등['std_spec'] = result3['Specificity'].std()
장년_적당_10_균등['mean_accu'] = result3['Accuracy'].mean()
장년_적당_10_균등['std_accu'] = result3['Accuracy'].std()
장년_적당_10_균등['mean_ppv'] = result3['PPV'].mean()
장년_적당_10_균등['std_ppv'] = result3['PPV'].std()
장년_적당_10_균등['mean_npv'] = result3['NPV'].mean()
장년_적당_10_균등['std_npv'] = result3['NPV'].std()
장년_적당_10_균등['mean_F1_score'] = result3['F_1 score'].mean()
장년_적당_10_균등['std_F1_score'] = result3['F_1 score'].std()

장년_적당_10_균등

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,77.035007,9.453669,99.999663,0.010647,97.730117,0.921807,99.996296,0.117121,97.547323,0.976277,86.694712,6.249572


In [119]:
장년_적당_10_균등.to_csv('../prophet_result/균등/10%/장년_적당_10_균등.csv')